
***[Open in Colab](https://colab.research.google.com/drive/1Ts0Y0pgVza3sosb2Cfp9_Um-HHDSwfTO?usp=sharing)***



***[Tools Documentation](https://python.langchain.com/docs/integrations/tools/)***

In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Succ

## Built in Tools

###DuckDuckGoSearch

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('top news in india today')
print(results)

Top - India - news - today ... info@truescoopnews.com ... Copyright © 2025 TrueScoop News . ... News 24 is your stop for those who are inquisitive enough to keep abreast with the ... Today News Headlines in India : Stay Updated with DLSNews24 ... News 24 is your stop for those who are inquisitive enough to keep abreast with the ... Today News Headlines in India : Stay Updated with DLSNews24 Explore the top business news today in India to stay informed about market developments. ... today for in -depth reporting and breaking news from India ... Explore the top business news today in India to stay informed about market developments. ... today for in -depth reporting and breaking news from India ...


In [ ]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### ShellTool

In [ ]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()
results = shell_tool.invoke('ls')
print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

### Using tool decorator

In [ ]:
from langchain_core.tools import tool

In [ ]:
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [ ]:
result = multiply.invoke({"a":3, "b":5})
print(result)

15


### Using Pydantic and Structured Tool

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [ ]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [ ]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Using Base Tool Class

In [ ]:
from langchain.tools import BaseTool
from typing import Type

In [ ]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [ ]:
multiply_tool = MultiplyTool()

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## ToolKits

In [ ]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [ ]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [ ]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
